# KNN으로 음수 가능 여부를 판단하기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2021)

## 1. Data

실습에서 사용할 데이터는 음수가 가능한지를 판단하는 데이터

### 1.1 Data Load

In [ ]:
water = pd.read_csv("/content/drive/MyDrive/MLStudy/ML Algorithm/06_KNN/water_potability.csv")

In [ ]:
data = water.drop(["Potability"], axis=1)
label = water["Potability"]

### 1.2 Data EDA
데이터와 변수들을 확인
count를 확인하면 count들이 다르기 때문에 몇몇 값이 비어있음을 유추 가능

In [ ]:
data.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,2785.000000,3276.000000,3276.000000,3276.000000,2495.000000,3276.000000,3276.000000,3114.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786
std,1.594320,32.879761,8768.570828,1.583085,41.416840,80.824064,3.308162,16.175008,0.780382
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000
25%,6.093092,176.850538,15666.690297,6.127421,307.699498,365.734414,12.065801,55.844536,3.439711
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028
75%,8.062066,216.667456,27332.762127,8.114887,359.950170,481.792304,16.557652,77.337473,4.500320
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000


값이 비어있는 데이터의 개수 확인

In [ ]:
data.isna().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
dtype: int64

### 1.3 Data Preprocess

빈 데이터를 제거하는 전처리를 수행
빈 데이터를 처리하는 방법은 row를 제거하는 법과 Column을 제거하는 방법이 있음

#### 1.3.1 row를 제거하는 방법

In [ ]:
na_cnt = data.isna().sum(axis=1)
na_cnt

0       1
1       1
2       1
3       0
4       0
       ..
3271    0
3272    2
3273    1
3274    1
3275    1
Length: 3276, dtype: int64

In [ ]:
drop_idx = na_cnt.loc[na_cnt > 0].index

In [ ]:
drop_idx

Int64Index([   0,    1,    2,    8,   11,   13,   14,   16,   18,   20,
            ...
            3247, 3252, 3258, 3259, 3260, 3266, 3272, 3273, 3274, 3275],
           dtype='int64', length=1265)

In [ ]:
drop_row = data.drop(drop_idx, axis=0)

In [ ]:
drop_row.shape

(2011, 9)

In [ ]:
data.shape

(3276, 9)

#### 1.3.2 column을 제거하는 방법

In [ ]:
na_cnt = data.isna().sum()
drop_cols = na_cnt.loc[na_cnt > 0].index

In [ ]:
drop_cols

Index(['ph', 'Sulfate', 'Trihalomethanes'], dtype='object')

In [ ]:
data = data.drop(drop_cols, axis=1)

### 1.4 Data Split
데이터를 Train, Test로 분할

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(
    data, label, train_size=0.7, random_state=2021
)

In [ ]:
print(f"train_data size: {len(train_label)}, {len(train_label)/len(data):.2f}")
print(f"test_data size: {len(test_label)}, {len(test_label)/len(data):.2f}")

train_data size: 2293, 0.70
test_data size: 983, 0.30


## 2. KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

### 2.1 Best Hyper Parameter
KNeighborsClassifier에서 탐색해야 할 argument
- n_neighbors
  - 몇 개의 이웃으로 예측할 것인지 정함
- p
  - 거리를 어떤 방식으로 계산할지 정함
    - 1. manhattan distance
    - 2. euclidean distance

In [ ]:
from sklearn.model_selection import GridSearchCV

#### 2.1.1 탐색 범위 선정

In [ ]:
params = {
    "n_neighbors": [i for i in range(1, 12, 2)],
    "p": [1, 2]
}

In [ ]:
params

{'n_neighbors': [1, 3, 5, 7, 9, 11], 'p': [1, 2]}

#### 2.1.2 탐색

In [ ]:
grid_cv = GridSearchCV(knn, param_grid=params, cv=3, n_jobs=-1)   # n_jobs = -1이면 모든 리소스를 사용해서 학습을 진행한다는 뜻

In [ ]:
grid_cv.fit(train_data, train_label)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11], 'p': [1, 2]})

#### 2.1.3 결과

In [ ]:
print(f"Best score of parameter search is {grid_cv.best_score_:.4f}")

Best score of parameter search is 0.5652


In [ ]:
grid_cv.best_params_

{'n_neighbors': 11, 'p': 1}

In [ ]:
print(f"Best parameter of best score is")
print(f"\t n_neighbors: {grid_cv.best_params_['n_neighbors']}")
print(f"\t p: {grid_cv.best_params_['p']}")

Best parameter of best score is
	 n_neighbors: 11
	 p: 1


#### 2.1.4 예측

In [ ]:
train_pred = grid_cv.best_estimator_.predict(train_data)
test_pred = grid_cv.best_estimator_.predict(test_data)

#### 2.1.5 평가

In [ ]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(train_label, train_pred)
test_acc = accuracy_score(test_label, test_pred)

In [ ]:
print(f"train accuracy is {train_acc:.4f}")
print(f"test accuracy is {test_acc:.4f}")

train accuracy is 0.6520
test accuracy is 0.5595


## 3. Scaling을 할 경우

### 3.1 Data Scaling
KNN은 거리를 기반으로 하는 알고리즘이기 때문에 데이터의 크기에 영향을 받음
e.g. 위에서 피쳐를 확인해보면 solids 피처의 값이 매우 크므로 거리 계신 시에 그 외의 값들이 거리에 큰 영향을 미치지 못하게 됨

따라서 Scaling을 진행해 크기를 맞춰주기

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
scaler.fit(train_data)

StandardScaler()

In [ ]:
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

### 3.2 탐색

In [ ]:
scaling_knn = KNeighborsClassifier()
scaling_grid_cv = GridSearchCV(scaling_knn, param_grid=params, n_jobs=-1)

In [ ]:
scaling_grid_cv.fit(scaled_train_data, train_label)

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11], 'p': [1, 2]})

In [ ]:
scaling_grid_cv.best_score_

0.587011825593896

In [ ]:
scaling_grid_cv.best_params_

{'n_neighbors': 9, 'p': 1}

### 3.3 평가

In [ ]:
scaling_train_pred = scaling_grid_cv.best_estimator_.predict(scaled_train_data)
scaling_test_pred = scaling_grid_cv.best_estimator_.predict(scaled_test_data)

In [ ]:
scaling_train_acc = accuracy_score(train_label, scaling_train_pred)
scaling_test_acc = accuracy_score(test_label, scaling_test_pred)

In [ ]:
print(f"Scaled data train accuracy is {scaling_train_acc:.4f}")
print(f"Scaled data test accuracy is {scaling_test_acc:.4f}")

Scaled data train accuracy is 0.6829
Scaled data test accuracy is 0.5799


## 4. 마무리

In [ ]:
print(f"test accuracy is {test_acc:.4f}")
print(f"Scaled data test accuracy is {scaling_test_acc:.4f}")

test accuracy is 0.5595
Scaled data test accuracy is 0.5799
